In [1]:
import pandas as pd
import os
import spacy 
nlp = spacy.load("en_core_web_sm")
from tqdm import tqdm
from collections import defaultdict
import statistics

# Score Storytelling

In [2]:
df_storytell = pd.read_csv('data/Valid_Storytelling_withVariable_VF_new3.csv')
#df_storytell = pd.read_csv('data/Test_Storytelling_withVariable_VF_new3.csv')

In [3]:
df_storytell.columns

Index(['PostSysID', 'AccountName', 'PostDate', 'PostID', 'PostUser',
       'PostText', 'MediaType', 'MediaLink', 'PlatformSysID', 'com_ct',
       'Angries', 'Comments Likes', 'Engagement', 'Hahas', 'Likes', 'Loves',
       'Replies', 'Retweets', 'Sads', 'Shares', 'Thankfuls', 'Views', 'Wows',
       'LIWC', 'Perspective', 'VADER', 'Intrigue_Prop_Change_Sent',
       'Conflictual_Event', 'Conflict_two_persos', 'Climax_increase',
       'first_person_sing', 'first_person_plur', 'second_person',
       'third_person_sing', 'third_person_plur', 'Binary_Entity',
       'Most_common_pronoun', 'Ratio_sentences_2_persos', 'EN_sent_subj',
       'Intrigue_Diversity', 'Intrigue_Sentence_Homogeneity',
       'Intrigue_Consistence', 'Causal_Coherence', 'Longest_Causal_Sequence',
       'IntraSentence_Causality', 'Presence_Dates', 'Logic_order_temporality',
       'Prop_temp_use', 'Logic_order_conjuguation',
       'Intrigue_Prop_Change_Sent.1', 'Conflictual_Event.1',
       'Conflict_two_persos.

## Score personnages

In [4]:
def score_perso(df):
    
    scores = []
    list_first_sing = list(df['first_person_sing'])
    list_first_plur = list(df['first_person_plur'])
    list_second = list(df['second_person'])
    list_third_sing = list(df['third_person_sing'])
    list_third_plur = list(df['third_person_plur'])
    list_binary_entity = list(df['Binary_Entity'])
    list_2_persos = list(df['Ratio_sentences_2_persos'])
    
    for i in tqdm(range(len(list_first_sing))):
        
        #First condition -- if no characters -- score = 0 here
        if list_first_sing[i] + list_first_plur[i] + list_second[i] + list_third_sing[i] + list_third_plur[i] == 0:
            score_perso = 0
            
        else: 
            score_perso = (0.650 * list_first_sing[i] + 0.674 * list_first_plur[i] + 0.722 * list_second[i] 
                           + 0.645 * list_third_sing[i] + 0.685 * list_third_plur[i] + 0.602 * list_binary_entity[i] + 0.685 * list_2_persos[i])
            score_perso = score_perso / (0.650 + 0.674 + 0.722 + 0.645 + 0.685 + 0.602 + 0.685)
        scores.append(score_perso)
    
    df['score_perso'] = scores
    return df

## Score intrigue

In [14]:
def score_intrigue(df):
    
    scores = []
    list_diversity = list(df['Intrigue_Diversity'])
    #list_homogeneity = list(df['new_Intrigue_Sentence_Homogeneity'])
    #list_heterogeneity = [(1 - x) for x in list_homogeneity]
    list_heterogeneity = list(df['new_Intrigue_Sentence_Heterogeneity'])
    list_consistence = list(df['new_Intrigue_Consistence'])
    list_coherence = list(df['Causal_Coherence'])
    list_causal_sequence = list(df['new_Longest_Causal_Sequence'])
    #list_causal_sequence = list(df['Longest_Causal_Sequence'])
    list_causality = list(df['new_IntraSentence_Causality'])
    list_dates = list(df['Presence_Dates'])
    list_logic_order = list(df['new_Logic_order_temporality'])
    list_temp_use = list(df['new_Prop_temp_use'])
    list_conjuguation = list(df['Logic_order_conjuguation'])
    list_prop_change_sent = list(df['new_Intrigue_Prop_Change_Sent'])
    
    for i in tqdm(range(len(list_diversity))):
        score_intrigue = (0.494 * list_diversity[i] + 0.259 * list_heterogeneity[i] + 0.347 * list_consistence[i] + 0.462 * list_coherence[i] + 
                          1.003 * list_causal_sequence[i] + 0.944 * list_causality[i] + 0.576 * list_dates[i] + 0.676 * list_logic_order[i] +
                          0.240 * list_temp_use[i] + 0.384 * list_conjuguation[i] + 0.484 * list_prop_change_sent[i])
        
        score_intrigue = score_intrigue / (0.494 + 0.259 + 0.347 + 0.462 + 1.003 + 0.944 + 0.576 + 0.676 + 0.240 + 0.384 + 0.484)
        
        scores.append(score_intrigue)
    
    df['score_intrigue'] = scores
    return df

## Score conflit

In [15]:
def score_conflict(df):
    
    scores = []
    list_event = list(df['Conflictual_Event'])
    list_two_persos = list(df['Conflict_two_persos'])
    list_climax_increase = list(df['new_Climax_increase'])
    list_neg_sentence = list(df['Neg_sentence_presence'])
    
    for i in tqdm(range(len(list_event))):
        score_intrigue = (0.751 * list_event[i] + 0.819 * list_two_persos[i] + 0.994 * list_climax_increase[i] + 0.521 * list_neg_sentence[i])
        score_intrigue = score_intrigue / (0.751 + 0.819 + 0.994 + 0.521)
        
        scores.append(score_intrigue)
        
    df['score_conflict'] = scores
    
    return df

## Score final

In [16]:
df_score = score_perso(df_storytell)
df_score = score_intrigue(df_score)
df_score = score_conflict(df_score)

100%|█████████████████████████████████████████████████████████████████████████| 4998/4998 [00:00<00:00, 4995979.84it/s]


In [17]:
df_score.columns

Index(['PostSysID', 'AccountName', 'PostDate', 'PostID', 'PostUser',
       'PostText', 'MediaType', 'MediaLink', 'PlatformSysID', 'com_ct',
       'Angries', 'Comments Likes', 'Engagement', 'Hahas', 'Likes', 'Loves',
       'Replies', 'Retweets', 'Sads', 'Shares', 'Thankfuls', 'Views', 'Wows',
       'LIWC', 'Perspective', 'VADER', 'Intrigue_Prop_Change_Sent',
       'Conflictual_Event', 'Conflict_two_persos', 'Climax_increase',
       'first_person_sing', 'first_person_plur', 'second_person',
       'third_person_sing', 'third_person_plur', 'Binary_Entity',
       'Most_common_pronoun', 'Ratio_sentences_2_persos', 'EN_sent_subj',
       'Intrigue_Diversity', 'Intrigue_Sentence_Homogeneity',
       'Intrigue_Consistence', 'Causal_Coherence', 'Longest_Causal_Sequence',
       'IntraSentence_Causality', 'Presence_Dates', 'Logic_order_temporality',
       'Prop_temp_use', 'Logic_order_conjuguation',
       'Intrigue_Prop_Change_Sent.1', 'Conflictual_Event.1',
       'Conflict_two_persos.

In [18]:
score_perso = (0.650 + 0.674 + 0.722 + 0.645 + 0.685 + 0.602 + 0.685) / 7
score_intrigue = (0.494 + 0.259 + 0.347 + 0.462 + 1.003 + 0.944 + 0.576 + 0.676 + 0.240 + 0.384 + 0.484) / 11
score_conflict = (0.751 + 0.819 + 0.994 + 0.521) / 4
df_score['Storytelling_score'] = df_score['score_perso'] + df_score['score_intrigue'] + df_score['score_conflict']
df_score['Storytelling_score_pondered'] = df_score['score_perso'].apply(lambda x: x*score_perso) + df_score['score_intrigue'].apply(lambda x: x*score_intrigue) + df_score['score_conflict'].apply(lambda x: x*score_conflict)

In [19]:
df_score.columns

Index(['PostSysID', 'AccountName', 'PostDate', 'PostID', 'PostUser',
       'PostText', 'MediaType', 'MediaLink', 'PlatformSysID', 'com_ct',
       'Angries', 'Comments Likes', 'Engagement', 'Hahas', 'Likes', 'Loves',
       'Replies', 'Retweets', 'Sads', 'Shares', 'Thankfuls', 'Views', 'Wows',
       'LIWC', 'Perspective', 'VADER', 'Intrigue_Prop_Change_Sent',
       'Conflictual_Event', 'Conflict_two_persos', 'Climax_increase',
       'first_person_sing', 'first_person_plur', 'second_person',
       'third_person_sing', 'third_person_plur', 'Binary_Entity',
       'Most_common_pronoun', 'Ratio_sentences_2_persos', 'EN_sent_subj',
       'Intrigue_Diversity', 'Intrigue_Sentence_Homogeneity',
       'Intrigue_Consistence', 'Causal_Coherence', 'Longest_Causal_Sequence',
       'IntraSentence_Causality', 'Presence_Dates', 'Logic_order_temporality',
       'Prop_temp_use', 'Logic_order_conjuguation',
       'Intrigue_Prop_Change_Sent.1', 'Conflictual_Event.1',
       'Conflict_two_persos.

In [20]:
def scores_total(df):
    
    Storytelling_mean, Storytelling_std1, Storytelling_std2, Storytelling_meanindiv, Storytelling_meanindiv_std1, Storytelling_mean_pond= [], [], [], [], [], []
    
    list_story = list(df['Storytelling_score'])
    list_perso = list(df['score_perso'])
    list_intrigue = list(df['score_intrigue'])
    list_conflict = list(df['score_conflict'])
    list_storypond = list(df['Storytelling_score_pondered'])
    
    mean_story, std_story = statistics.mean(list_story), statistics.stdev(list_story)
    mean_perso, std_perso = statistics.mean(list_perso), statistics.stdev(list_perso)
    mean_intrigue, std_intrigue = statistics.mean(list_intrigue), statistics.stdev(list_intrigue)
    mean_conflict, std_conflict = statistics.mean(list_conflict), statistics.stdev(list_conflict)
    mean_pond, std_pond = statistics.mean(list_storypond), statistics.stdev(list_storypond)
    
    for i in tqdm(range(len(list_story))):
        
        if list_story[i] > mean_story:
            Storytelling_mean.append(1)
        else:
            Storytelling_mean.append(0)
            
        if list_story[i] > mean_story + std_story :
            Storytelling_std1.append(1)
        else:
            Storytelling_std1.append(0)
            
        if list_story[i] > mean_story + 2 * std_story :
            Storytelling_std2.append(1)
        else:
            Storytelling_std2.append(0)
            
        if list_storypond[i] > mean_pond :
            Storytelling_mean_pond.append(1)
        else:
            Storytelling_mean_pond.append(0)
            
        cond1, cond2, cond3, cond4, cond5, cond6 = False, False, False, False, False, False
        if list_perso[i] > mean_perso:
            cond1 = True
        if list_intrigue[i] > mean_intrigue:
            cond2 = True
        if list_conflict[i] > mean_conflict:
            cond3 = True
        if list_perso[i] > mean_perso + std_perso:
            cond4 = True
        if list_intrigue[i] > mean_intrigue + std_intrigue:
            cond5 = True
        if list_conflict[i] > mean_conflict + std_conflict:
            cond1 = True
            
        if cond1 and cond2 and cond3:
            Storytelling_meanindiv.append(1)
        else:
            Storytelling_meanindiv.append(0)
            
        if cond4 and cond5 and cond6:
            Storytelling_meanindiv_std1.append(1)
        else:
            Storytelling_meanindiv_std1.append(0)

    df['Storytelling_mean'] = Storytelling_mean
    df['Storytelling_std1'] = Storytelling_std1
    df['Storytelling_std2'] = Storytelling_std2
    df['Storytelling_meanindiv'] = Storytelling_meanindiv
    df['Storytelling_meanindiv_std1'] = Storytelling_meanindiv_std1
    df['Storytelling_meanindiv_pondered'] = Storytelling_mean_pond
    
    return df

In [21]:
df_score_f = scores_total(df_score)

100%|█████████████████████████████████████████████████████████████████████████| 4998/4998 [00:00<00:00, 1997059.29it/s]


In [22]:
df_score_f.columns

Index(['PostSysID', 'AccountName', 'PostDate', 'PostID', 'PostUser',
       'PostText', 'MediaType', 'MediaLink', 'PlatformSysID', 'com_ct',
       'Angries', 'Comments Likes', 'Engagement', 'Hahas', 'Likes', 'Loves',
       'Replies', 'Retweets', 'Sads', 'Shares', 'Thankfuls', 'Views', 'Wows',
       'LIWC', 'Perspective', 'VADER', 'Intrigue_Prop_Change_Sent',
       'Conflictual_Event', 'Conflict_two_persos', 'Climax_increase',
       'first_person_sing', 'first_person_plur', 'second_person',
       'third_person_sing', 'third_person_plur', 'Binary_Entity',
       'Most_common_pronoun', 'Ratio_sentences_2_persos', 'EN_sent_subj',
       'Intrigue_Diversity', 'Intrigue_Sentence_Homogeneity',
       'Intrigue_Consistence', 'Causal_Coherence', 'Longest_Causal_Sequence',
       'IntraSentence_Causality', 'Presence_Dates', 'Logic_order_temporality',
       'Prop_temp_use', 'Logic_order_conjuguation',
       'Intrigue_Prop_Change_Sent.1', 'Conflictual_Event.1',
       'Conflict_two_persos.

In [23]:
df_score.to_csv('202403_Valid_Storytelling_VariablesAndScores_vFinal.csv', index=False)

# Parasocial

In [1]:
import pandas as pd
import os
import spacy 
nlp = spacy.load("en_core_web_sm")
from tqdm import tqdm
from collections import defaultdict
import statistics
import numpy as np

In [15]:
#df_para = pd.read_csv('data/202403_Valid_Parasocial_withVariable_VF.csv')
df_para = pd.read_csv('data/202403_Test_Parasocial_withVariable_VF.csv')

## Score Cognitive

In [3]:
def score_attention(df):

    scores = []
    list_vb_avg_min = list(df['vb_avg_min'])
    list_vb_avg_avg = list(df['vb_avg_avg'])
    list_imply = list(df['avg_post_imply'])
    #list_engage_rate = list(df['engage_rate'])   # Normalizing by number of comments for each posts
    list_engage_rate = list(df['new_engage_rate'])
    list_post = list(df['nb_comments'])
    
    for i in tqdm(range(len(list_vb_avg_min))):
        score = list_vb_avg_min[i] + list_vb_avg_avg[i] + list_imply[i] + (list_engage_rate[i] / list_post[i])
        score = score / 4
        scores.append(score)
    
    df['score_attention'] = scores
    return df

In [4]:
def score_comprehension(df):

    scores = []
    list_similarity = list(df['semantic_similarity'])
    list_predicted = list(df['comment_predicted'])
    list_repeat = list(df['new_avg_repeat'])
    
    for i in tqdm(range(len(list_similarity))):
        score = (list_similarity[i] + list_predicted[i] + list_repeat[i]) / 3
        scores.append(score)
    
    df['score_comprehension'] = scores
    return df

In [5]:
def score_prior(df):

    scores = []
    list_past = list(df['past_exp'])
    list_compare = list(df['comparison'])
    list_implied = list(df['new_comments_implied'])
    
    for i in tqdm(range(len(list_past))):
        score = (list_past[i] + list_compare[i] + list_implied[i]) / 3
        scores.append(score)
    
    df['score_prior'] = scores
    return df

In [6]:
def score_evaluation(df):

    scores = []
    list_weak = list(df['weak_subj'])
    list_strong = list(df['strong_subj'])
    list_neutral = list(df['neutral'])  ### 1- neutrality since we seek for non neutrality
    list_objective = list(df['objectivity'])  ### 1- objectivity since we seek for non objectivity
    
    for i in tqdm(range(len(list_weak))):
        score = (list_weak[i] + list_strong[i] + (1-list_neutral[i]) + (1-list_objective[i])) / 4
        scores.append(score)
    
    df['score_evaluation'] = scores
    return df

In [7]:
def score_anticipatory(df):

    scores = []
    list_evol = list(df['brand_evolution'])
    list_condi = list(df['brand_conditional'])
    
    for i in tqdm(range(len(list_evol))):
        score = (list_evol[i] + list_condi[i]) / 2
        scores.append(score)
    
    df['score_anticipatory'] = scores
    return df

In [8]:
def score_relation(df):

    scores = []
    list_proj = list(df['contextual_projection'])
    list_obsess = list(df['obsession'])
    list_action = list(df['link_my_action'])
    list_mention = list(df['direct_mention'])
    
    for i in tqdm(range(len(list_proj))):
        score = list_proj[i] + list_obsess[i] + list_action[i] + list_mention[i]
        score = score / 4
        scores.append(score)
    
    df['score_relation'] = scores
    return df

## Score affective

In [9]:
def score_affective(df):
    
    list_sent = list(df['sentiment'])
    list_intens = list(df['intensity'])
    list_empat = list(df['empathy'])
    list_same = list(df['same_intensity'])
    list_conta = list(df['sentiment_contagion']) ### If condition for -1 0 or 1 here -- si -1 = 0 dans le score | si 0 = 0.5 | si 1 = 1
    list_repeat = list(df['nb_letter_repeat'])  ### Normalize by max value == 2.5
    norm_repeat = max(list_repeat)
    list_upper = list(df['nb_uppercase'])
    
    scores, scores_sympathy, scores_empathy, scores_contagion = [], [], [] , []
    for i in tqdm(range(len(list_sent))):
        ##Condition for sentiment_contagion:
        score_conta = 0
        if list_conta[i] == 0:
            score_conta = 0.5
        elif list_conta[i] == 1:
            score_conta = 1
            
        score_sympathy = (list_sent[i] + list_intens[i]) / 2
        scores_sympathy.append(score_sympathy)
        score_empathy = (list_empat[i] + list_same[i]) / 2
        scores_empathy.append(score_empathy)
        score_contagion = (score_conta + list_repeat[i]/norm_repeat + list_upper[i])/3
        scores_contagion.append(score_contagion)
        score = (score_sympathy + score_empathy + score_contagion)  / 3
        scores.append(score)
        
    df['score_sympathy'] = scores_sympathy
    df['score_empathy'] = scores_empathy
    df['score_contagion'] = scores_contagion
    df['score_affective'] = scores
    
    return df

## Score behavior

In [10]:
def score_behav(df):
    
    
    list_likes = list(df['Likes'])
    list_comments = list(df['nb_comments'])
    
    ##creating normalized score for comments here : 
    comment_likes = []
    for i in range(len(list_likes)):
        comment_likes.append(list_comments[i] / list_likes[i])
    
    norm_comm = max(comment_likes)                        #Normalizing by max value
    list_emojis = list(df['emojis'])                      #Normalizing by max value
    norm_emo = max(list_emojis)
    
    list_nbemojis = list(df['new_nb_comments_emojis'])
    list_write = list(df['new_time_to_write'])
    list_mistakes = list(df['nb_mistakes'])               #Normalizing by max value
    norm_mistakes = max(list_mistakes) 
    list_slangs = list(df['nb_slangs'])
    list_answers = list(df['nb_answers'])
    list_intent = list(df['intent'])
    
    scores, scores_nonverbal, scores_paraverbal, scores_intentions = [], [], [], []
    for i in tqdm(range(len(comment_likes))):
        
        score_nonverbal = ((comment_likes[i]/norm_comm) + (list_emojis[i]/norm_emo) + list_nbemojis[i]) / 3
        scores_nonverbal.append(score_nonverbal)
        score_paraverbal = (list_write[i] + (list_mistakes[i]/norm_mistakes) + list_slangs[i]) / 3
        scores_paraverbal.append(score_paraverbal)
        score_intentions = (list_answers[i] + list_intent[i]) / 2
        scores_intentions.append(score_intentions)
        
        score = (score_nonverbal + score_paraverbal + score_intentions)/3
        scores.append(score)
        
    df['score_nonverbal'] = scores_nonverbal
    df['score_paraverbal'] = scores_paraverbal
    df['score_intentions'] = scores_intentions
    df['score_behavioral'] = scores
    
    return df

## Score finals

In [11]:
df_score = score_attention(df_para)
df_score = score_comprehension(df_score)
df_score = score_prior(df_score)
df_score = score_evaluation(df_score)
df_score = score_anticipatory(df_score)
df_score = score_relation(df_score)

100%|██████████████████████████████████████████████████████████████████████████████████████| 5000/5000 [00:00<?, ?it/s]


In [13]:
score_cognitive = [(list(df_score['score_attention'])[i] + list(df_score['score_prior'])[i] + list(df_score['score_comprehension'])[i] + 
                   list(df_score['score_evaluation'])[i] + list(df_score['score_anticipatory'])[i] + list(df_score['score_relation'])[i])/6 for i in range(len(list(df_score['score_attention'])))]
df_score['score_cognitive'] = score_cognitive

In [ ]:
df_score = score_affective(df_score)
df_score = score_behav(df_score)

In [15]:
df_score['Parasocial_score'] = df_score['score_cognitive'] + df_score['score_affective'] + df_score['score_behavioral']

In [16]:
def scores_total(df):
    
    Para_mean, Para_std1, Para_std2, Para_meanindiv, Para_full_indiv= [], [], [], [], []
    #Cognitive
    list_att = list(df['score_attention'])
    mean_att = statistics.mean(list_att)
    list_prior = list(df['score_prior'])
    mean_prior = statistics.mean(list_prior)
    list_comp = list(df['score_comprehension'])
    mean_comp = statistics.mean(list_comp)
    list_eval = list(df['score_evaluation'])
    mean_eval = statistics.mean(list_eval)
    list_antic = list(df['score_anticipatory'])
    mean_antic = statistics.mean(list_antic)
    list_rel = list(df['score_relation'])
    mean_rel = statistics.mean(list_rel)
    list_cog = list(df_score['score_cognitive'])
    mean_cog = statistics.mean(list_cog)
    
    #Affect
    list_symp = list(df['score_sympathy'])
    mean_symp = statistics.mean(list_symp)
    list_emp = list(df['score_empathy'])
    mean_emp = statistics.mean(list_emp)
    list_conta = list(df['score_contagion'])
    mean_conta = statistics.mean(list_conta)
    list_affect = list(df['score_affective'])
    mean_affect = statistics.mean(list_affect)
    #Behavior
    list_nonver = list(df['score_nonverbal'])
    mean_nonver = statistics.mean(list_nonver)
    list_para = list(df['score_paraverbal'])
    mean_para = statistics.mean(list_para)
    list_intent = list(df['score_intentions'])
    mean_intent = statistics.mean(list_intent)
    list_behav = list(df['score_behavioral'])
    mean_behav = statistics.mean(list_behav)
    #Total
    list_parasocial = list(df['Parasocial_score'])
    mean_parasocial, std_parasocial = statistics.mean(list_parasocial), statistics.stdev(list_parasocial)
    
    
    for i in tqdm(range(len(list_parasocial))):
        #Creating the parascore if the aggregated is superior to the mean
        if list_parasocial[i] > mean_parasocial:
            Para_mean.append(1)
        else:
            Para_mean.append(0)
            
         #Creating the parascore if the aggregated is superior to the mean + 1 std deviation
        if list_parasocial[i] > mean_parasocial + std_parasocial :
            Para_std1.append(1)
        else:
            Para_std1.append(0)
            
         #Creating the parascore if the aggregated is superior to the mean + 2 std deviation
        if list_parasocial[i] > mean_parasocial + 2 * std_parasocial :
            Para_std2.append(1)
        else:
            Para_std2.append(0)
            
        #Creating the score if all variables are above their mean
        cond1_, cond2_, cond3_ = False, False, False
        if list_cog[i] > mean_cog:
            cond1_ = True
        if list_affect[i] > mean_affect:
            cond2_ = True
        if list_behav[i] > mean_behav:
            cond3_ = True
            
        if cond1_ and cond2_ and cond3_:
            Para_meanindiv.append(1)
        else:
            Para_meanindiv.append(0)
            
        cond1, cond2, cond3, cond4, cond5, cond6 = False, False, False, False, False, False
        cond7, cond8, cond9, cond10, cond11, cond12 = False, False, False, False, False, False
        if list_att[i] > mean_att:
            cond1 = True
        if list_prior[i] > mean_prior:
            cond2 = True
        if list_comp[i] > mean_comp:
            cond3 = True
        if list_eval[i] > mean_eval:
            cond4 = True
        if list_antic[i] > mean_antic:
            cond5 = True
        if list_rel[i] > mean_rel:
            cond6 = True
        if list_symp[i] > mean_symp:
            cond7 = True
        if list_emp[i] > mean_emp:
            cond8 = True
        if list_conta[i] > mean_conta:
            cond9 = True
        if list_nonver[i] > mean_nonver:
            cond10 = True
        if list_intent[i] > mean_intent:
            cond11 = True
        if list_para[i] > mean_para:
            cond12 = True

        if cond1 and cond2 and cond3 and cond4 and cond5 and cond6 and cond7 and cond8 and cond9 and cond10 and cond11 and cond12:
            Para_full_indiv.append(1)
        else:
            Para_full_indiv.append(0)
            
    df['Parasocial_mean'] = Para_mean
    df['Parasocial_std1'] = Para_std1
    df['Parasocial_std2'] = Para_std2
    df['Parasocial_meanindiv'] = Para_meanindiv
    df['Parasocial_fullindiv'] = Para_full_indiv
    
    return df

In [17]:
df_score_f = scores_total(df_score)

100%|█████████████████████████████████████████████████████████████████████████| 5000/5000 [00:00<00:00, 1248676.39it/s]


In [18]:
df_score_f.columns

Index(['PostSysID', 'AccountName', 'PostDate', 'PostID', 'PostUser',
       'PostText', 'MediaType', 'MediaLink', 'PlatformSysID', 'com_ct',
       'Angries', 'Comments Likes', 'Engagement', 'Hahas', 'Likes', 'Loves',
       'Replies', 'Retweets', 'Sads', 'Shares', 'Thankfuls', 'Views', 'Wows',
       'LIWC', 'Perspective', 'VADER', 'vb_avg_min', 'vb_avg_avg',
       'avg_previous', 'avg_post_imply', 'semantic_similarity',
       'comment_predicted', 'past_exp', 'impliedsituation', 'comparison',
       'weak_subj', 'strong_subj', 'brand_evolution', 'brand_conditional',
       'contextual_projection', 'obsession', 'link_my_action',
       'direct_mention', 'sentiment', 'intensity', 'empathy', 'same_intensity',
       'sentiment_contagion', 'nb_letter_repeat', 'nb_uppercase',
       'nb_comments', 'nb_slangs', 'nb_mistakes', 'time_to_write',
       'nb_answers', 'intent', 'emojis', 'nb_comments_emojis', 'neutral',
       'objectivity', 'avg_repeat', 'new_nb_comments_emojis',
       'new_

In [19]:
df_score_f.head()

,PostSysID,AccountName,PostDate,PostID,PostUser,PostText,MediaType,MediaLink,PlatformSysID,com_ct,...,score_nonverbal,score_paraverbal,score_intentions,score_behavioral,Parasocial_score,Parasocial_mean,Parasocial_std1,Parasocial_std2,Parasocial_meanindiv,Parasocial_fullindiv
0,15230642,Machine Gun Kelly,2016-09-28,https://www.instagram.com/p/BK6jXBBBt0s/?taken...,machinegunkelly,day 1. see you tomorrow Spokane. #AlphaOmegaTour,video/mp4,https://instagram.fyhu1-1.fna.fbcdn.net/vp/392...,3,337.0,...,0.053523,0.054667,0.048077,0.091463,0.613075,0,0,0,0,0
1,9930130,Russ,2017-09-09,https://www.instagram.com/p/BY1oGG8hmHM/?taken...,russ,Performed at the Kith fashion show the other n...,image,https://instagram.fyhu1-1.fna.fbcdn.net/vp/4a9...,3,1688.0,...,0.053523,0.054667,0.048077,0.093396,0.712455,1,0,0,0,0
2,17030309,Don Diablo,2017-06-03,https://www.instagram.com/p/BU5IRcWBVj1/?taken...,dondiablo,The Flying Dutch! One day three shows. This wa...,image,https://instagram.fyhu1-1.fna.fbcdn.net/vp/c04...,3,110.0,...,0.053523,0.054667,0.048077,0.097340,0.790759,1,1,0,1,0
3,15230614,Machine Gun Kelly,2016-10-09,https://www.instagram.com/p/BLW8UTnh7gr/?taken...,machinegunkelly,happy 4th anniversary to our first album.,image,https://instagram.fyhu1-1.fna.fbcdn.net/vp/1c9...,3,758.0,...,0.053523,0.054667,0.048077,0.065848,0.699464,1,0,0,0,0
4,15230458,Machine Gun Kelly,2016-12-09,https://www.instagram.com/p/BNzghDPhF9p/?taken...,machinegunkelly,smells like teen spirit in NYC today 😈,image,https://instagram.fyhu1-1.fna.fbcdn.net/vp/86e...,3,314.0,...,0.053523,0.054667,0.048077,0.094335,0.657101,1,0,0,0,0


In [20]:
df_score.to_csv('202403_Valid_Parasocial_VariablesAndScores_vFinal.csv', index=False)